In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pvdeg
import dask.array as da
import dask.dataframe as dd
import xarray as xr
import os
import socket
import pandas as pd

In [3]:
# Start HPC cluster

local = {'manager': 'local',
       'n_workers': 5,}

kestrel = {
    'manager': 'slurm',
    'n_jobs': 2,  # Max number of nodes used for parallel processing
    'cores': 100,
    'processes': 50,
    'memory': '256GB',
    'account': 'pvfem',
    'queue': 'short',
    'walltime': '4:00:00',
    'scheduler_options':{'host':socket.gethostname()}
    }

client = pvdeg.geospatial.start_dask(hpc=local)


/home/sayala/.local/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33235 instead
  warnings.warn(


Dashboard: http://127.0.0.1:33235/status


In [4]:
work_dir = '/projects/pvsoiling/pvdeg/analysis/world_map/standoff_fine'
data_dir = '/projects/pvsoiling/pvdeg/analysis/world_map/data'

In [5]:
meta_north = pd.read_csv(f'{data_dir}/meta_pvgis_north_3300.csv',index_col=0)
weather_north = xr.open_dataset(f'{data_dir}/weather_ds_north_3300.nc')
weather_north = weather_north.sel(gid=meta_north.index)

weather_north = weather_north.assign_coords({'gid': meta_north.index})
weather_north = weather_north.chunk(chunks={"time": -1, "gid": 100})
weather_north = weather_north.unify_chunks()

In [9]:
jsonfolder = {'jsonfolder':'/home/sayala/jsons'}
type(jsonfolder)

dict

In [10]:
geo = {'func': pvdeg.standards.vertical_POA,
       'weather_ds': weather_north,
       'meta_df': meta_north,
       'func_kwargs': jsonfolder}

vertical_res = pvdeg.geospatial.analysis(**geo)

2024-06-26 16:38:30,935 - distributed.worker - WARNING - Compute Failed
Key:       ('calc_block-4d72863a0ebd25772578fd2feb2ef910', 16, 0)
State:     executing
Function:  execute_task
args:      ((<function map_blocks.<locals>._wrapper at 0x7f313cf3bd30>, <function calc_block at 0x7f30d0e2eaf0>, [(<class 'xarray.core.dataset.Dataset'>, (<class 'dict'>, [['temp_air', (('gid', 'time'), array([[-17.26, -17.72, -18.17, ..., -15.89, -16.35, -16.8 ],
       [-16.36, -16.42, -16.47, ..., -16.2 , -16.26, -16.31],
       [-16.51, -16.85, -17.19, ..., -15.5 , -15.83, -16.17],
       ...,
       [-37.84, -37.23, -36.62, ..., -39.66, -39.05, -38.44],
       [-29.31, -26.96, -24.6 , ..., -36.39, -34.03, -31.67],
       [-30.33, -27.91, -25.5 , ..., -37.57, -35.15, -32.74]]), {})], ['relative_humidity', (('gid', 'time'), array([[60.9 , 60.5 , 60.11, ..., 62.09, 61.7 , 61.3 ],
       [56.49, 56.21, 55.93, ..., 57.34, 57.06, 56.78],
       [46.33, 50.3 , 54.28, ..., 34.41, 38.38, 42.35],
       ...,
  

AttributeError: 'Index' object has no attribute 'is_monotonic'

In [ ]:
vertical_res['']

In [ ]:
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.PlateCarree(), frameon=True)
ax.patch.set_visible(True)
ax.set_extent([-180, 180, -85, 85], ccrs.PlateCarree())

shapename = "admin_0_countries"
states_shp = shpreader.natural_earth(
    resolution="110m", category="cultural", name=shapename
)

ax.add_geometries(
    shpreader.Reader(states_shp).geometries(),
    ccrs.PlateCarree(),
    facecolor="none",
    edgecolor="gray",
    linewidth=0.5,
)

cmap = "Spectral_r"
size = 0.75
vertical_res.plot.scatter(x='longitude', y='latitude', hue='lcoe_nom', cmap=cmap, s=size, linewidths=0, add_colorbar=False, ax=ax) # vmin=0, vmax=14,

plt.xlim([-180,180])
plt.ylim([-90,90])

In [ ]:
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader


data_projection=ccrs.PlateCarree()
map_projection=ccrs.Orthographic(0, 90)

fig = plt.figure(figsize=(10, 5))
ax = fig.add_axes([0, 0, 1, 1], projection=map_projection, frameon=True)
ax.patch.set_visible(True)

shapename = "admin_0_countries"
states_shp = shpreader.natural_earth(
    resolution="110m", category="cultural", name=shapename
)

ax.add_geometries(
    shpreader.Reader(states_shp).geometries(),
    data_projection,
    facecolor="none",
    edgecolor="gray",
    linewidth=0.5,
)

cmap = "Spectral_r"
size = 0.75
vertical_res.plot.scatter(x='longitude', y='latitude', hue='x', cmap=cmap, s=size, linewidths=0, vmin=0, vmax=14, add_colorbar=False, ax=ax, transform=data_projection)

